In [ ]:
import openai
client = OpenAI(api_key=OPENAI_API_KEY)

def get_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return np.array(response.data[0].embedding, dtype=np.float32)


In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_embedding(text):
    return embedding_model.encode(text, convert_to_numpy=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 769.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import faiss
import wikipediaapi
import numpy as np
import os
import requests
from sentence_transformers import SentenceTransformer

# Configuration
OPENROUTER_API_KEY = "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"
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='MyApp/1.0 (contact@example.com)'
)

def get_wikipedia_content(query):
    """Fetch Wikipedia content with error handling."""
    try:
        page = wiki_wiki.page(query)
        return page.summary if page.exists() else None
    except Exception as e:
        print(f"Wikipedia error: {str(e)}")
        return None

# Initialize embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# FAISS index configuration
d = 384  # Matches model embedding size
index = faiss.IndexFlatL2(d)
document_store = []

def add_to_faiss(text):
    """Process and store text embeddings."""
    if not text:
        return
    try:
        embedding = embedding_model.encode(text, convert_to_numpy=True)
        embedding = embedding.astype(np.float32)  # Ensure correct dtype
        index.add(np.expand_dims(embedding, axis=0))  # Proper shape
        document_store.append(text)
    except Exception as e:
        print(f"FAISS error: {str(e)}")

def retrieve_relevant_document(query):
    """Semantic search with error handling."""
    try:
        query_embedding = embedding_model.encode(query, convert_to_numpy=True)
        query_embedding = query_embedding.astype(np.float32).reshape(1, -1)
        distances, indices = index.search(query_embedding, k=1)
        return document_store[indices[0][0]]
    except (IndexError, ValueError) as e:
        print(f"Retrieval error: {str(e)}")
        return None

# OpenRouter API Client
def query_openrouter(prompt, model="mistralai/mistral-7b-instruct"):
    """Improved API request with comprehensive error handling."""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": "https://github.com/your-repository",  # Required header
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API Request failed: {str(e)}")
        return None

# Main execution flow
if __name__ == "__main__":
    # Seed with AI content
    ai_content = get_wikipedia_content("Vaaranam Aayiram")
    if ai_content:
        add_to_faiss(ai_content)
        print("Successfully indexed AI content")
    else:
        print("Failed to retrieve AI content")

    # User query
    user_query = "What is cast of Vaaranam Aayiram movie ? and their character names"

    # Retrieve context
    context = retrieve_relevant_document(user_query)

    if context:
        # Construct prompt
        prompt = f"""Answer the question using only the provided context.

        Context:
        {context}

        Question: {user_query}

        Answer:"""

        # Get response
        response = query_openrouter(prompt)

        if response and "choices" in response:
            print("\nFinal Answer:")
            print(response["choices"][0]["message"]["content"])
        else:
            print("Failed to get valid response from API")
    else:
        print("No relevant context found")


Successfully indexed AI content

Final Answer:
 The cast of Vaaranam Aayiram movie is as follows:

1. Suriya - He plays the dual lead roles of the father, Krishnan, and the son, who is also named Krishnan.
2. Simran - Her character name is not specified in the context provided.
3. Sameera Reddy - She plays the role of Soudamini, the love interest of the son Krishnan.
4. Ramya - Her character name is not specified in the context provided.


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import wikipediaapi
import requests
from sentence_transformers import SentenceTransformer

# Configuration
OPENROUTER_API_KEY = "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"
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='ContentAssistant/1.0 (contact@example.com)'
)

def get_wikipedia_content(query):
    """Fetch Wikipedia content with error handling."""
    try:
        page = wiki_wiki.page(query)
        return page.summary if page.exists() else None
    except Exception as e:
        print(f"Wikipedia error: {str(e)}")
        return None

def query_openrouter(prompt):
    """OpenRouter API client with enhanced error handling."""
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "HTTP-Referer": "https://github.com/content-assistant",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "mistralai/mistral-7b-instruct",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.5
    }

    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=20
        )
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"API Error: {str(e)}")
        return None

def process_query(topic, user_question):
    """Main processing function for Gradio interface"""
    # Get Wikipedia content
    content = get_wikipedia_content(topic)
    if not content:
        return f"Error: Could not retrieve information for '{topic}' from Wikipedia."

    try:
        # Generate response using OpenRouter API
        prompt = f"""Answer concisely using only this context. If unsure, say you don't know.

        Context: {content}

        Question: {user_question}

        Answer in 2-3 sentences:"""

        response = query_openrouter(prompt)

        # Prepare output combining Wikipedia content, user query, and AI-generated answer
        ai_response = response['choices'][0]['message']['content'] if response else "Error: Failed to get a valid response from OpenRouter."

        output = f"Wikipedia Content:\n\n{content}\n\nUser Query:\n\n{user_question}\n\nAI Response:\n\n{ai_response}"
        return output

    except Exception as e:
        return f"Processing error: {str(e)}"

# Create Gradio interface
interface = gr.Interface(
    fn=process_query,
    inputs=[
        gr.Textbox(label="Topic", placeholder="Enter a topic (e.g., Artificial Intelligence, Earth, etc.)"),
        gr.Textbox(label="Your Question", placeholder="Ask anything about the topic...")
    ],
    outputs=gr.Textbox(label="Result"),
    title="🌍 Wikipedia Content Assistant",
    description="Access information on any topic from Wikipedia and receive AI-generated answers.",
    examples=[
        ["Artificial Intelligence", "What are its applications in modern technology?"],
        ["Earth", "What is the composition of Earth's atmosphere?"],
        ["Python (programming language)", "What are its key features?"]
    ],
    allow_flagging="never"
)

# Launch the app
if __name__ == "__main__":
    interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aef0c4718938e0cf35.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
